# Machine Learning Trading Bot

In this Challenge, you’ll assume the role of a financial advisor at one of the top five financial advisory firms in the world. Your firm constantly competes with the other major firms to manage and automatically trade assets in a highly dynamic environment. In recent years, your firm has heavily profited by using computer algorithms that can buy and sell faster than human traders.

The speed of these transactions gave your firm a competitive advantage early on. But, people still need to specifically program these systems, which limits their ability to adapt to new data. You’re thus planning to improve the existing algorithmic trading systems and maintain the firm’s competitive advantage in the market. To do so, you’ll enhance the existing trading signals with machine learning algorithms that can adapt to new data.

## Instructions:

Use the starter code file to complete the steps that the instructions outline. The steps for this Challenge are divided into the following sections:

* Establish a Baseline Performance

* Tune the Baseline Trading Algorithm

* Evaluate a New Machine Learning Classifier

* Create an Evaluation Report

#### Establish a Baseline Performance

In this section, you’ll run the provided starter code to establish a baseline performance for the trading algorithm. To do so, complete the following steps.

Open the Jupyter notebook. Restart the kernel, run the provided cells that correspond with the first three steps, and then proceed to step four. 

1. Import the OHLCV dataset into a Pandas DataFrame.

2. Generate trading signals using short- and long-window SMA values. 

3. Split the data into training and testing datasets.

4. Use the `SVC` classifier model from SKLearn's support vector machine (SVM) learning method to fit the training data and make predictions based on the testing data. Review the predictions.

5. Review the classification report associated with the `SVC` model predictions. 

6. Create a predictions DataFrame that contains columns for “Predicted” values, “Actual Returns”, and “Strategy Returns”.

7. Create a cumulative return plot that shows the actual returns vs. the strategy returns. Save a PNG image of this plot. This will serve as a baseline against which to compare the effects of tuning the trading algorithm.

8. Write your conclusions about the performance of the baseline trading algorithm in the `README.md` file that’s associated with your GitHub repository. Support your findings by using the PNG image that you saved in the previous step.

#### Tune the Baseline Trading Algorithm

In this section, you’ll tune, or adjust, the model’s input features to find the parameters that result in the best trading outcomes. (You’ll choose the best by comparing the cumulative products of the strategy returns.) To do so, complete the following steps:

1. Tune the training algorithm by adjusting the size of the training dataset. To do so, slice your data into different periods. Rerun the notebook with the updated parameters, and record the results in your `README.md` file. Answer the following question: What impact resulted from increasing or decreasing the training window?

> **Hint** To adjust the size of the training dataset, you can use a different `DateOffset` value&mdash;for example, six months. Be aware that changing the size of the training dataset also affects the size of the testing dataset.

2. Tune the trading algorithm by adjusting the SMA input features. Adjust one or both of the windows for the algorithm. Rerun the notebook with the updated parameters, and record the results in your `README.md` file. Answer the following question: What impact resulted from increasing or decreasing either or both of the SMA windows?

3. Choose the set of parameters that best improved the trading algorithm returns. Save a PNG image of the cumulative product of the actual returns vs. the strategy returns, and document your conclusion in your `README.md` file.

#### Evaluate a New Machine Learning Classifier

In this section, you’ll use the original parameters that the starter code provided. But, you’ll apply them to the performance of a second machine learning model. To do so, complete the following steps:

1. Import a new classifier, such as `AdaBoost`, `DecisionTreeClassifier`, or `LogisticRegression`. (For the full list of classifiers, refer to the [Supervised learning page](https://scikit-learn.org/stable/supervised_learning.html) in the scikit-learn documentation.)

2. Using the original training data as the baseline model, fit another model with the new classifier.

3. Backtest the new model to evaluate its performance. Save a PNG image of the cumulative product of the actual returns vs. the strategy returns for this updated trading algorithm, and write your conclusions in your `README.md` file. Answer the following questions: Did this new model perform better or worse than the provided baseline model? Did this new model perform better or worse than your tuned trading algorithm?

#### Create an Evaluation Report

In the previous sections, you updated your `README.md` file with your conclusions. To accomplish this section, you need to add a summary evaluation report at the end of the `README.md` file. For this report, express your final conclusions and analysis. Support your findings by using the PNG images that you created.


In [34]:
# Imports
import sys
import platform

import pandas as pd
import numpy as np
import sklearn as sk
from pathlib import Path
                             
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset

from sklearn import svm                                 #  support vector machine (SVM) learning method
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import classification_report, balanced_accuracy_score, confusion_matrix, f1_score
from imblearn.metrics import classification_report_imbalanced

import hvplot.pandas
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import pickle

In [35]:
# Report Technologies
print(f'Python Platform: {platform.platform()}')
print(f'Python {sys.version}')
print()
print(f'Pandas {pd.__version__}')
print(f'Numpy {np.__version__}')
print(f'Scikit-Learn {sk.__version__}')
print(f'Seaborn {sns.__version__}')

Python Platform: macOS-13.0.1-arm64-arm-64bit
Python 3.9.15 | packaged by conda-forge | (main, Nov 22 2022, 08:52:10) 
[Clang 14.0.6 ]

Pandas 1.5.1
Numpy 1.21.5
Scikit-Learn 1.1.3
Seaborn 0.12.0


---

## Establish a Baseline Performance

In this section, you’ll run the provided starter code to establish a baseline performance for the trading algorithm. To do so, complete the following steps.

Open the Jupyter notebook. Restart the kernel, run the provided cells that correspond with the first three steps, and then proceed to step four. 


### Step 1: import the OHLCV dataset into a Pandas DataFrame.

In [3]:
# Import the OHLCV dataset into a Pandas Dataframe
ohlcv_df = pd.read_csv(
    Path('./Resources/emerging_markets_ohlcv.csv'), 
    index_col='date', 
    infer_datetime_format=True, 
    parse_dates=True
)

# Review the DataFrame
display(ohlcv_df.shape)
display(ohlcv_df.head(3))

(4323, 5)

,open,high,low,close,volume
date,,,,,
2015-01-21 09:30:00,23.83,23.83,23.83,23.83,100
2015-01-21 11:00:00,23.98,23.98,23.98,23.98,100
2015-01-22 15:00:00,24.42,24.42,24.42,24.42,100


In [4]:
# Filter the date index and close columns
signals_df = ohlcv_df.loc[:, ['close']]

# Use the pct_change function to generate  returns from close prices
signals_df['Actual Returns'] = signals_df['close'].pct_change()

# Drop all NaN values from the DataFrame
signals_df = signals_df.dropna()

# Review the DataFrame
display(signals_df.head(3))
display(signals_df.tail(3))

,close,Actual Returns
date,,
2015-01-21 11:00:00,23.98,0.006295
2015-01-22 15:00:00,24.42,0.018349
2015-01-22 15:15:00,24.44,0.000819


,close,Actual Returns
date,,
2021-01-22 13:45:00,33.42,0.002099
2021-01-22 14:30:00,33.47,0.001496
2021-01-22 15:45:00,33.44,-0.000896


## Step 2: Generate trading signals using short- and long-window SMA values. 

In [5]:
# Set the short window and long window
short_window = 4
long_window = 100

# Generate the fast and slow simple moving averages (4 and 100 days, respectively)
signals_df['SMA_Fast'] = signals_df['close'].rolling(window=short_window).mean()
signals_df['SMA_Slow'] = signals_df['close'].rolling(window=long_window).mean()

signals_df = signals_df.dropna()

# Review the DataFrame
display(signals_df.head(3))
display(signals_df.tail(3))

,close,Actual Returns,SMA_Fast,SMA_Slow
date,,,,
2015-04-02 14:45:00,24.92,0.000000,24.9175,24.3214
2015-04-02 15:00:00,24.92,0.000000,24.9200,24.3308
2015-04-02 15:15:00,24.94,0.000803,24.9250,24.3360


,close,Actual Returns,SMA_Fast,SMA_Slow
date,,,,
2021-01-22 13:45:00,33.42,0.002099,33.3850,30.48745
2021-01-22 14:30:00,33.47,0.001496,33.3775,30.53085
2021-01-22 15:45:00,33.44,-0.000896,33.4200,30.57495


In [6]:
# Initialize the new Signal column
signals_df['Signal'] = 0.0

# When Actual Returns are greater than or equal to 0, generate signal to buy stock long
signals_df.loc[(signals_df['Actual Returns'] >= 0), 'Signal'] = 1

# When Actual Returns are less than 0, generate signal to sell stock short
signals_df.loc[(signals_df['Actual Returns'] < 0), 'Signal'] = -1

# Review the DataFrame
display(signals_df.head(3))
display(signals_df.tail(3))

,close,Actual Returns,SMA_Fast,SMA_Slow,Signal
date,,,,,
2015-04-02 14:45:00,24.92,0.000000,24.9175,24.3214,1.0
2015-04-02 15:00:00,24.92,0.000000,24.9200,24.3308,1.0
2015-04-02 15:15:00,24.94,0.000803,24.9250,24.3360,1.0


,close,Actual Returns,SMA_Fast,SMA_Slow,Signal
date,,,,,
2021-01-22 13:45:00,33.42,0.002099,33.3850,30.48745,1.0
2021-01-22 14:30:00,33.47,0.001496,33.3775,30.53085,1.0
2021-01-22 15:45:00,33.44,-0.000896,33.4200,30.57495,-1.0


In [7]:
signals_df['Signal'].value_counts()

 1.0    2368
-1.0    1855
Name: Signal, dtype: int64

In [8]:
# Calculate the strategy returns and add them to the signals_df DataFrame
signals_df['Strategy Returns'] = signals_df['Actual Returns'] * signals_df['Signal'].shift()

# Review the DataFrame
display(signals_df.head(3))
display(signals_df.tail(3))

,close,Actual Returns,SMA_Fast,SMA_Slow,Signal,Strategy Returns
date,,,,,,
2015-04-02 14:45:00,24.92,0.000000,24.9175,24.3214,1.0,NaN
2015-04-02 15:00:00,24.92,0.000000,24.9200,24.3308,1.0,0.000000
2015-04-02 15:15:00,24.94,0.000803,24.9250,24.3360,1.0,0.000803


,close,Actual Returns,SMA_Fast,SMA_Slow,Signal,Strategy Returns
date,,,,,,
2021-01-22 13:45:00,33.42,0.002099,33.3850,30.48745,1.0,0.002099
2021-01-22 14:30:00,33.47,0.001496,33.3775,30.53085,1.0,0.001496
2021-01-22 15:45:00,33.44,-0.000896,33.4200,30.57495,-1.0,-0.000896


In [9]:
# Plot Strategy Returns to examine performance
(1 + signals_df['Strategy Returns']).cumprod().hvplot()

:Curve   [date]   (Strategy Returns)

### Step 3: Split the data into training and testing datasets.

In [10]:
# Assign a copy of the sma_fast and sma_slow columns to a features DataFrame called X
X = signals_df[['SMA_Fast', 'SMA_Slow']].shift().dropna()

# Review the DataFrame
X.head(3)

,SMA_Fast,SMA_Slow
date,,
2015-04-02 15:00:00,24.9175,24.3214
2015-04-02 15:15:00,24.9200,24.3308
2015-04-02 15:30:00,24.9250,24.3360


In [11]:
# Alternate features set split
# Assign a copy of the sma_fast and sma_slow columns to a features DataFrame called X
Xa = signals_df[['SMA_Fast', 'SMA_Slow']].shift().dropna()

# Review the DataFrame
Xa.head(3)

,SMA_Fast,SMA_Slow
date,,
2015-04-02 15:00:00,24.9175,24.3214
2015-04-02 15:15:00,24.9200,24.3308
2015-04-02 15:30:00,24.9250,24.3360


In [12]:
# plot features
X.hvplot()

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [13]:
# Create the target set selecting the Signal column and assiging it to y
y = signals_df['Signal']

# Review the value counts
display(y.shape)
display(y.value_counts())

(4223,)

 1.0    2368
-1.0    1855
Name: Signal, dtype: int64

In [14]:
# Alternate target set split
# Create the target set selecting the Signal column and assiging it to y
ya = signals_df['Signal']

# Review the value counts
display(ya.shape)
display(ya.value_counts())

(4223,)

 1.0    2368
-1.0    1855
Name: Signal, dtype: int64

In [15]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

2015-04-02 15:00:00


In [16]:
# Alternate
# Select the start of the training period
training_begin_a = Xa.index.min()

# Display the training begin date
print(training_begin_a)

2015-04-02 15:00:00


In [17]:
# Select the ending period for the training data with an offset of 3 months 
training_end = X.index.min() + DateOffset(months=3)

# Display the training end date
print(training_end)

2015-07-02 15:00:00


In [18]:
# Alternate
# Select the ending period for the training data with an offset of 64 months
training_end_a = Xa.index.min() + DateOffset(months=24)

# Display the training end date
print(training_end_a)

2017-04-02 15:00:00


In [19]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
display(X_train.shape)
display(X_train.head(3))

(128, 2)

,SMA_Fast,SMA_Slow
date,,
2015-04-02 15:00:00,24.9175,24.3214
2015-04-02 15:15:00,24.9200,24.3308
2015-04-02 15:30:00,24.9250,24.3360


In [20]:
# Alternate
# Generate the X_train and y_train DataFrames
Xa_train = Xa.loc[training_begin_a:training_end_a]
ya_train = ya.loc[training_begin_a:training_end_a]

# Review the X_train DataFrame
display(Xa_train.shape)
display(Xa_train.head(3))

(1428, 2)

,SMA_Fast,SMA_Slow
date,,
2015-04-02 15:00:00,24.9175,24.3214
2015-04-02 15:15:00,24.9200,24.3308
2015-04-02 15:30:00,24.9250,24.3360


In [21]:
# Plot the training set
X_train.hvplot()

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [22]:
# Alternate
# Plot the training set
Xa_train.hvplot()

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [23]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]

# Review the X_test DataFrame
display(X_test.shape)
display(X_test.head(3))

(4092, 2)

,SMA_Fast,SMA_Slow
date,,
2015-07-06 10:00:00,24.1250,25.0919
2015-07-06 10:45:00,23.9700,25.0682
2015-07-06 14:15:00,23.8475,25.0458


# inspect features datasets' lengths, i.e. review the split

In [24]:
# inspect the lengths of the full features dataset and of split data
display(len(X))           # 4222
display(len(X_train))     # 128
display(len(X_test))      # 4092

4222

128

4092

In [25]:
# Generate the X_test and y_test DataFrames
Xa_test = Xa.loc[training_end_a+DateOffset(hours=1):]
ya_test = ya.loc[training_end_a+DateOffset(hours=1):]

# Review the X_test DataFrame
display(Xa_test.shape)
display(Xa_test.head(3))

(2794, 2)

,SMA_Fast,SMA_Slow
date,,
2017-04-03 11:15:00,22.8650,22.68825
2017-04-03 11:30:00,22.8375,22.69285
2017-04-03 13:30:00,22.8350,22.69775


In [26]:
# plot the test set
X_test.hvplot()

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [27]:
# Alternate
# plot the test set
Xa_test.hvplot()

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [28]:
# Scale the features DataFrames

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# review shape 
display(X_train_scaled.shape)
display(X_test_scaled.shape)

(128, 2)

(4092, 2)

In [29]:
# Alternate - 24 months train
# Scale the features DataFrames

# Create a StandardScaler instance
scaler_a = StandardScaler()

# Apply the scaler model to fit the X-train data
Xa_scaler = scaler_a.fit(Xa_train)

# Transform the X_train and X_test DataFrames using the X_scaler
Xa_train_scaled = Xa_scaler.transform(Xa_train)
Xa_test_scaled = Xa_scaler.transform(Xa_test)

# review shape 
display(Xa_train_scaled.shape)
display(Xa_test_scaled.shape)

(1428, 2)

(2794, 2)

### Step 4: Use the `SVC` classifier model from SKLearn's support vector machine (SVM) learning method to fit the training data and make predictions based on the testing data. Review the predictions.

In [30]:
# From SVM, instantiate SVC classifier model instance
SEED = 42
svm_model = svm.SVC(random_state = SEED)
 
# Fit the model to the data using the training data
svm_model = svm_model.fit(X_train_scaled, y_train)
 
# Use the testing data to make the model predictions
svm_predictions = svm_model.predict(X_test_scaled)

# Review the model's predicted values
svm_predictions[:50]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [36]:
# Save SVM 3 month model - pickle method
# Save to file in the current working directory
pkl_svm_model = 'svm_model.pkl'
with open(pkl_svm_model, 'wb') as file:
    pickle.dump(svm_model, file)


In [31]:
# Save SVM 3 month model
# Set the model's file path
file_path = Path('./Resources_models/svm3m.h5')

# Export your model to a HDF5 file
svm_model.save(file_path)

# Save model as JSON
svm_model_json = svm_model.to_json()

file_path = Path('./Resources_models/svm3m.json')
with open(file_path, 'w') as json_file:
    json_file.write(svm_model_json)
    
# Save the model weights
file_path = './Resources_models/svm3mj.h5'
svm_model.save_weights('./Resources_models/svm3mj.h5')

AttributeError: 'SVC' object has no attribute 'save'

In [32]:
# Alternate - 24 month train
# From SVM, instantiate SVC classifier model instance
SEED = SEED
svm_a_model = svm.SVC(random_state = SEED)
 
# Fit the model to the data using the training data
svm_a_model = svm_a_model.fit(Xa_train_scaled, ya_train)
 
# Use the testing data to make the model predictions
svm_a_predictions = svm_a_model.predict(Xa_test_scaled)

# Review the model's predicted values
svm_a_predictions[:50]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [33]:
# Save SVM 24 month model
# Set the model's file path
file_path = Path('./Resources_models/svm24m.h5')

# Export your model to a HDF5 file
svm_a_model.save(file_path)

# Save model as JSON
svm_a_model_json = svm_a_model.to_json()

file_path = Path('./Resources_models/svm24m.json')
with open(file_path, 'w') as json_file:
    json_file.write(svm_a_model_json)
    
# Save the model weights
file_path = './Resources_models/svm24mj.h5'
svm_a_model.save_weights('./Resources_models/svm24mj.h5')

AttributeError: 'SVC' object has no attribute 'save'

### Step 5: Review the classification report associated with the `SVC` model predictions. 

In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
svm_testing_report = classification_report(y_test, svm_predictions)

# Print the classification report
print(svm_testing_report )

In [ ]:
# Calculate the balanced_accuracy score of the model
svm_balanced_accuracy_score = balanced_accuracy_score(y_test, svm_predictions)

# Generate a confusion matrix for the model
svm_matrix = confusion_matrix(y_test, svm_predictions)

# Print the confusion matrix for the original test data
print(svm_matrix)

In [ ]:
# Calculate the classification report for the model
svm_imbalance_testing_report = classification_report_imbalanced(y_test, svm_predictions)

In [ ]:
# Adding classes names for better interpretation
classes_names = ['Negative (-1)','Positive (+1)']
cm = pd.DataFrame(confusion_matrix(y_test, svm_predictions), 
                  columns=classes_names, index = classes_names)
                  
# Seaborn's heatmap to better visualize the confusion matrix
sns.heatmap(cm, annot = True, fmt = 'd', cmap='Blues', linewidth =.5, linecolor = 'black')
# Print the balanced_accuracy score of the classifier19 model
print('\nbalanced accuracy score:', svm_balanced_accuracy_score)
# Print the classification report for the classifier19 model
print('\n',svm_imbalance_testing_report) 

In [ ]:
# Alternate - 24 month train
# Use a classification report to evaluate the model using the predictions and testing data
svm_a_testing_report = classification_report(ya_test, svm_a_predictions)

# Print the classification report
print(svm_a_testing_report )

In [ ]:
# Alternate - 24 month train
# Calculate the balanced_accuracy score of the model
svm_a_balanced_accuracy_score = balanced_accuracy_score(ya_test, svm_a_predictions)

# Generate a confusion matrix for the model
svm_a_matrix = confusion_matrix(ya_test, svm_a_predictions)

# Print the confusion matrix for the original test data
print(svm_a_matrix)

In [ ]:
# Alternate - 24 month train
# Calculate the classification report for the model
svm_a_imbalance_testing_report = classification_report_imbalanced(ya_test, svm_a_predictions)

In [ ]:
# Alternate - 24 month train
# Adding classes names for better interpretation
classes_names = ['Negative (-1)','Positive (+1)']
cm = pd.DataFrame(confusion_matrix(ya_test, svm_a_predictions), 
                  columns=classes_names, index = classes_names)
                  
# Seaborn's heatmap to better visualize the confusion matrix
sns.heatmap(cm, annot = True, fmt = 'd', cmap='Blues', linewidth =.5, linecolor = 'black');
# Print the balanced_accuracy score of the classifier19 model
print('\nbalanced accuracy score:', svm_a_balanced_accuracy_score)
# Print the classification report for the classifier19 model
print('\n',svm_a_imbalance_testing_report) 

### Step 6: Create a predictions DataFrame that contains columns for “Predicted” values, “Actual Returns”, and “Strategy Returns”.

In [ ]:
# Create a new empty predictions DataFrame.

# Create a predictions DataFrame
predictions_df = pd.DataFrame(index = X_test.index)

# Add the SVM model predictions to the DataFrame
predictions_df['Predicted'] = svm_predictions

# Add the actual returns to the DataFrame
predictions_df['Actual Returns'] = signals_df['Actual Returns']

# Add the strategy returns to the DataFrame
predictions_df['Strategy Returns'] = (predictions_df['Predicted'] * predictions_df['Actual Returns'])

# Review the DataFrame
display(predictions_df.head(3))
display(predictions_df.tail(3))

In [ ]:
# Create a new empty predictions DataFrame.

# Create a predictions DataFrame
predictions_a_df = pd.DataFrame(index = Xa_test.index)

# Add the SVM model predictions to the DataFrame
predictions_a_df['Predicted'] = svm_a_predictions

# Add the actual returns to the DataFrame
predictions_a_df['Actual Returns'] = signals_df['Actual Returns']

# Add the strategy returns to the DataFrame
predictions_a_df['Strategy Returns'] = (predictions_a_df['Predicted'] * predictions_a_df['Actual Returns'])

# Review the DataFrame
display(predictions_a_df.head(3))
display(predictions_a_df.tail(3))

### Step 7: Create a cumulative return plot that shows the actual returns vs. the strategy returns. Save a PNG image of this plot. This will serve as a baseline against which to compare the effects of tuning the trading algorithm.

In [ ]:
# Plot the actual returns versus the strategy returns
plot_1 = (1 + predictions_df[['Actual Returns', 'Strategy Returns']]).cumprod().hvplot(title = 'SVM learning method - 3 month training')

In [ ]:
# Alternate
# Plot the actual returns versus the strategy returns
plot_1a = (1 + predictions_a_df[['Actual Returns', 'Strategy Returns']]).cumprod().hvplot(title = 'SVM learning method - 24 month training')
plot_1 + plot_1a

---

## Tune the Baseline Trading Algorithm

## Step 6: Use an Alternative ML Model and Evaluate Strategy Returns

In this section, you’ll tune, or adjust, the model’s input features to find the parameters that result in the best trading outcomes. You’ll choose the best by comparing the cumulative products of the strategy returns.

### Step 1: Tune the training algorithm by adjusting the size of the training dataset. 

To do so, slice your data into different periods. Rerun the notebook with the updated parameters, and record the results in your `README.md` file. 

Answer the following question: What impact resulted from increasing or decreasing the training window?

### Step 2: Tune the trading algorithm by adjusting the SMA input features. 

Adjust one or both of the windows for the algorithm. Rerun the notebook with the updated parameters, and record the results in your `README.md` file. 

Answer the following question: What impact resulted from increasing or decreasing either or both of the SMA windows?

### Step 3: Choose the set of parameters that best improved the trading algorithm returns. 

Save a PNG image of the cumulative product of the actual returns vs. the strategy returns, and document your conclusion in your `README.md` file.

---

## Evaluate a New Machine Learning Classifier

In this section, you’ll use the original parameters that the starter code provided. But, you’ll apply them to the performance of a second machine learning model. 

### Step 1:  Import a new classifier, such as `AdaBoost`, `DecisionTreeClassifier`, or `LogisticRegression`. (For the full list of classifiers, refer to the [Supervised learning page](https://scikit-learn.org/stable/supervised_learning.html) in the scikit-learn documentation.)

In [ ]:
# Import a new classifier from SKLearn
# from sklearn.ensemble import AdaBoostClassifier

### Step 2: Using the original training data as the baseline model, fit another model with the new classifier.

In [ ]:
# Initiate the model instance
boost_model = AdaBoostClassifier(random_state = SEED) 

# Fit the model using the training data
boost_model = boost_model.fit(X_train_scaled, y_train)

# Use the testing dataset to generate the predictions for the new model
boost_pred = boost_model.predict(X_test_scaled)

# Review the model's predicted values
boost_pred[:50]

In [ ]:
# Alternate
# Initiate the model instance
boost_a_model = AdaBoostClassifier(random_state = SEED)

# Fit the model using the training data
boost_a_model = boost_a_model.fit(Xa_train_scaled, ya_train)

# Use the testing dataset to generate the predictions for the new model
boost_a_pred = boost_a_model.predict(Xa_test_scaled)

# Review the model's predicted values
boost_a_pred[:50]

### Step 3: Backtest the new model to evaluate its performance. 

Save a PNG image of the cumulative product of the actual returns vs. the strategy returns for this updated trading algorithm, and write your conclusions in your `README.md` file. 

Answer the following questions: 
Did this new model perform better or worse than the provided baseline model? 
Did this new model perform better or worse than your tuned trading algorithm?

In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
boost_testing_report = classification_report(y_test, boost_pred)

# Print the classification report
print(boost_testing_report )

In [ ]:
# Calculate the balanced_accuracy score of the model
boost_balanced_accuracy_score = balanced_accuracy_score(y_test, boost_pred)

# Generate a confusion matrix for the model
boost_matrix = confusion_matrix(y_test, boost_pred)

# Print the confusion matrix for the original test data
print(boost_matrix)

In [ ]:
# Calculate the classification report for the model
boost_imbalance_testing_report = classification_report_imbalanced(y_test, boost_pred)

In [ ]:
# Adding classes names for better interpretation
classes_names = ['Negative (-1)','Positive (+1)']
cm = pd.DataFrame(confusion_matrix(y_test, boost_pred), 
                  columns=classes_names, index = classes_names)
                  
# Seaborn's heatmap to better visualize the confusion matrix
sns.heatmap(cm, annot = True, fmt = 'd', cmap='Greens', linewidth =.5, linecolor = 'black');
# Print the balanced_accuracy score of the classifier19 model
print('\nbalanced accuracy score:', boost_balanced_accuracy_score)
# Print the classification report for the classifier19 model
print('\n', boost_imbalance_testing_report) 

In [ ]:
# Alternate - 24 month train
# Use a classification report to evaluate the model using the predictions and testing data
boost_a_testing_report = classification_report(ya_test, boost_a_pred)

# Print the classification report
print(boost_a_testing_report )

In [ ]:
# Alternate - 24 month train
# Calculate the balanced_accuracy score of the model
boost_a_balanced_accuracy_score = balanced_accuracy_score(ya_test, boost_a_pred)

# Generate a confusion matrix for the model
boost_a_matrix = confusion_matrix(ya_test, boost_a_pred)

# Print the confusion matrix for the original test data
print(boost_a_matrix)

In [ ]:
# Alternate - 24 month train
# Calculate the classification report for the model
boost_a_imbalance_testing_report = classification_report_imbalanced(ya_test, boost_a_pred)

In [ ]:
# Alternate - 24 month train
# Adding classes names for better interpretation
classes_names = ['Negative (-1)','Positive (+1)']
cm = pd.DataFrame(confusion_matrix(ya_test, boost_a_pred), 
                  columns=classes_names, index = classes_names)
                  
# Seaborn's heatmap to better visualize the confusion matrix
sns.heatmap(cm, annot = True, fmt = 'd', cmap='Greens', linewidth =.5, linecolor = 'black');
# Print the balanced_accuracy score of the classifier19 model
print('\nbalanced accuracy score:', boost_a_balanced_accuracy_score)
# Print the classification report for the classifier19 model
print('\n', boost_a_imbalance_testing_report) 

In [ ]:
# Create a new empty predictions DataFrame.

# Create a predictions DataFrame
boost_predictions_df = pd.DataFrame(index = X_test.index)

# Add the SVM model predictions to the DataFrame
boost_predictions_df['Predicted'] = boost_pred

# Add the actual returns to the DataFrame
boost_predictions_df['Actual Returns'] = signals_df['Actual Returns']

# Add the strategy returns to the DataFrame
boost_predictions_df['Strategy Returns'] = (boost_predictions_df['Predicted'] * boost_predictions_df['Actual Returns'])

# Review the DataFrame
display(boost_predictions_df.head(3))
display(boost_predictions_df.tail(3))

In [ ]:
# Plot the actual returns versus the strategy returns
plot_2 = (1 + boost_predictions_df[['Actual Returns', 'Strategy Returns']]).cumprod().hvplot(title = 'AdaBoostClassifier - 3 month training')

In [ ]:
# Create a new empty predictions DataFrame.

# Create a predictions DataFrame
boost_a_predictions_df = pd.DataFrame(index = Xa_test.index)

# Add the SVM model predictions to the DataFrame
boost_a_predictions_df['Predicted'] = boost_a_pred

# Add the actual returns to the DataFrame
boost_a_predictions_df['Actual Returns'] = signals_df['Actual Returns']

# Add the strategy returns to the DataFrame
boost_a_predictions_df['Strategy Returns'] = (boost_a_predictions_df['Predicted'] * boost_a_predictions_df['Actual Returns'])

# Review the DataFrame
display(boost_a_predictions_df.head(3))
display(boost_a_predictions_df.tail(3))

In [ ]:
# Alternate
# Plot the actual returns versus the strategy returns
plot_2a = (1 + boost_a_predictions_df[['Actual Returns', 'Strategy Returns']]).cumprod().hvplot(title = 'AdaBoostClassifier - 24 month training')
plot_2 + plot_2a

In [ ]:
# instantiate the model (using the default parameters)
logreg_model = LogisticRegression(random_state = SEED)

# Fit the model using the training data
logreg_model = logreg_model.fit(X_train_scaled, y_train)

# Use the testing dataset to generate the predictions for the new model
logreg_pred = logreg_model.predict(X_test_scaled)

# Review the model's predicted values
logreg_pred[:50]

In [ ]:
# Alternate
# instantiate the model (using the default parameters)
logreg_a_model = LogisticRegression(random_state = SEED)

# Fit the model using the training data
logreg_a_model = logreg_a_model.fit(Xa_train_scaled, ya_train)

# Use the testing dataset to generate the predictions for the new model
logreg_a_pred = logreg_a_model.predict(Xa_test_scaled)

# Review the model's predicted values
logreg_a_pred[:50]

In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
logreg_testing_report = classification_report(y_test, logreg_pred)

# Print the classification report
print(logreg_testing_report)

In [ ]:
# Calculate the balanced_accuracy score of the model
logreg_balanced_accuracy_score = balanced_accuracy_score(y_test, logreg_pred)

# Generate a confusion matrix for the model
logreg_matrix = confusion_matrix(y_test, logreg_pred)

# Print the confusion matrix for the original test data
print(logreg_matrix)

In [ ]:
# Calculate the classification report for the model
logreg_imbalance_testing_report = classification_report_imbalanced(y_test, logreg_pred)

In [ ]:
# Adding classes names for better interpretation
classes_names = ['Negative (-1)','Positive (+1)']
cm = pd.DataFrame(confusion_matrix(y_test, logreg_pred), 
                  columns=classes_names, index = classes_names)
                  
# Seaborn's heatmap to better visualize the confusion matrix
sns.heatmap(cm, annot = True, fmt = 'd', cmap='Reds', linewidth =.5, linecolor = 'black');
# Print the balanced_accuracy score of the classifier19 model
print('\nbalanced accuracy score:', logreg_balanced_accuracy_score)
# Print the classification report for the classifier19 model
print('\n', logreg_imbalance_testing_report) 

In [ ]:
# Alternate
# Use a classification report to evaluate the model using the predictions and testing data
logreg_a_testing_report = classification_report(ya_test, logreg_a_pred)

# Print the classification report
print(logreg_a_testing_report )

In [ ]:
# Alternate - 24 month train
# Calculate the balanced_accuracy score of the model
logreg_a_balanced_accuracy_score = balanced_accuracy_score(ya_test, logreg_a_pred)

# Generate a confusion matrix for the model
logreg_a_matrix = confusion_matrix(ya_test, logreg_a_pred)

# Print the confusion matrix for the original test data
print(logreg_a_matrix)

In [ ]:
# Alternate - 24 month train
# Calculate the classification report for the model
logreg_a_imbalance_testing_report = classification_report_imbalanced(ya_test, logreg_a_pred)

In [ ]:
# Alternate - 24 month train
# Adding classes names for better interpretation
classes_names = ['Negative (-1)','Positive (+1)']
cm = pd.DataFrame(confusion_matrix(ya_test, logreg_a_pred), 
                  columns=classes_names, index = classes_names)
                  
# Seaborn's heatmap to better visualize the confusion matrix
sns.heatmap(cm, annot = True, fmt = 'd', cmap='Reds', linewidth =.5, linecolor = 'black');
# Print the balanced_accuracy score of the classifier19 model
print('\nbalanced accuracy score:', logreg_a_balanced_accuracy_score)
# Print the classification report for the classifier19 model
print('\n', logreg_a_imbalance_testing_report) 

In [ ]:
# Create a new empty predictions DataFrame.

# Create a predictions DataFrame
logreg_predictions_df = pd.DataFrame(index = X_test.index)

# Add the SVM model predictions to the DataFrame
logreg_predictions_df['Predicted'] = logreg_pred

# Add the actual returns to the DataFrame
logreg_predictions_df['Actual Returns'] = signals_df['Actual Returns']

# Add the strategy returns to the DataFrame
logreg_predictions_df['Strategy Returns'] = (logreg_predictions_df['Predicted'] * logreg_predictions_df['Actual Returns'])

# Review the DataFrame
display(logreg_predictions_df.head(3))
display(logreg_predictions_df.tail(3))

In [ ]:
# Plot the actual returns versus the strategy returns
plot_3 = (1 + logreg_predictions_df[['Actual Returns', 'Strategy Returns']]).cumprod().hvplot(title = 'LogisticRegression - 3 month training')

In [ ]:
# Create a new empty predictions DataFrame.

# Create a predictions DataFrame
logreg_a_predictions_df = pd.DataFrame(index = Xa_test.index)

# Add the SVM model predictions to the DataFrame
logreg_a_predictions_df['Predicted'] = logreg_a_pred

# Add the actual returns to the DataFrame
logreg_a_predictions_df['Actual Returns'] = signals_df['Actual Returns']

# Add the strategy returns to the DataFrame
logreg_a_predictions_df['Strategy Returns'] = (logreg_a_predictions_df['Predicted'] * logreg_a_predictions_df['Actual Returns'])

# Review the DataFrame
display(logreg_a_predictions_df.head(3))
display(logreg_a_predictions_df.tail(3))

In [ ]:
# Alternate
# Plot the actual returns versus the strategy returns
plot_3a = (1 + logreg_a_predictions_df[['Actual Returns', 'Strategy Returns']]).cumprod().hvplot(title = 'LogisticRegression - 24 month training')
plot_3 + plot_3a

___
# SUPPLEMENTAL EXPERIMENTATION FOLLOWS
___

In [ ]:
# Instantiate KNN model
classifier_model = KNeighborsClassifier()

# fit or 'train' KNN
classifier_model = classifier_model.fit(X_train_scaled, y_train)

# Use the testing dataset to generate the predictions for the new model
classifier_pred = classifier_model.predict(X_test_scaled)

# Review the model's predicted values
display(type(classifier_pred))
classifier_pred[:50]

In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
classifier_testing_report = classification_report(y_test, classifier_pred)

# Print the classification report
print(classifier_testing_report)

In [ ]:
# Calculate the balanced_accuracy score of the model
classifier_balanced_accuracy_score = balanced_accuracy_score(y_test, classifier_pred)

# Generate a confusion matrix for the model
classifier_matrix = confusion_matrix(y_test, classifier_pred)

# Print the confusion matrix for the original test data
print(classifier_matrix)

In [ ]:
# Calculate the classification report for the model
classifier_imbalance_testing_report = classification_report_imbalanced(y_test, classifier_pred)

In [ ]:
# Adding classes names for better interpretation
classes_names = ['Negative (-1)','Positive (+1)']
cm = pd.DataFrame(confusion_matrix(y_test, classifier_pred), 
                  columns=classes_names, index = classes_names)
                  
# Seaborn's heatmap to better visualize the confusion matrix
sns.heatmap(cm, annot = True, fmt = 'd', cmap='Reds', linewidth =.5, linecolor = 'black');
# Print the balanced_accuracy score of the classifier19 model
print('\nbalanced accuracy score:', classifier_balanced_accuracy_score)
# Print the classification report for the classifier19 model
print('\n', classifier_imbalance_testing_report) 

# Finding the Best K for KNN Classification
### create a for loop and run models that have from 1 to X neighbors. At each interaction, calculate the f1 and plot the number of Ks along with the f1 result

In [ ]:
# create an empty set called 'f1s'
f1s = []

# Calculating f1 score for K values between 1 and 40
for i in range(1, 40):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train_scaled, y_train)
    pred_i = knn.predict(X_test_scaled)
    # using average = 'weighted' to calculate a weighted average for the classes 
    f1s.append(f1_score(y_test, pred_i, average='weighted'))

# plot the f1 values to find the best K

In [ ]:
# plot the f1s
plt.figure(figsize=(12, 6))
plt.plot(range(1, 40), f1s, color='red', linestyle='dashed', marker='o',
         markerfacecolor='blue', markersize = 10)
plt.title('F1 Score K Value')
plt.xlabel('K Value')
plt.ylabel('F1 Score')

In [ ]:
# New f1 plot, less data (i.e. shorter range)
plt.figure(figsize=(12, 6))
plt.plot(range(1, 10), f1s[:9], color='red', linestyle='dashed', marker='o',
         markerfacecolor='blue', markersize=10)
plt.title('F1 Score K Value')
plt.xlabel('K Value')
plt.ylabel('F1 Score')

In [ ]:
# obtain the highest f1 using the built-in max()
print(max(f1s))                       # 0.5094344985062002
# convert the list into a NumPy array and use argmax() to get highest index of that point
print(np.array(f1s).argmax())         # 1, remember zero indexing

In [ ]:
# Instantiate new KNN model
classifier2_model = KNeighborsClassifier(n_neighbors = 1)     

# fit or 'train' KNN
classifier2_model = classifier2_model.fit(X_train_scaled, y_train)

# Use the testing dataset to generate the predictions for the new model
classifier2_pred = classifier2_model.predict(X_test_scaled)

# Review the model's predicted values
display(type(classifier2_pred))
classifier2_pred[:50]

In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
classifier2_testing_report = classification_report(y_test, classifier2_pred)

# Print the classification report
print(classifier2_testing_report)

In [ ]:
# Calculate the balanced_accuracy score of the model
classifier2_balanced_accuracy_score = balanced_accuracy_score(y_test, classifier2_pred)

# Generate a confusion matrix for the model
classifier2_matrix = confusion_matrix(y_test, classifier2_pred)

# Print the confusion matrix for the original test data
print(classifier2_matrix)

In [ ]:
# Calculate the classification report for the model
classifier2_imbalance_testing_report = classification_report_imbalanced(y_test, classifier2_pred)

In [ ]:
# Adding classes names for better interpretation
classes_names = ['Negative (-1)','Positive (+1)']
cm = pd.DataFrame(confusion_matrix(y_test, classifier2_pred), 
                  columns=classes_names, index = classes_names)
                  
# Seaborn's heatmap to better visualize the confusion matrix
sns.heatmap(cm, annot = True, fmt = 'd', cmap='Reds', linewidth =.5, linecolor = 'black');
# Print the balanced_accuracy score of the classifier19 model
print('\nbalanced accuracy score:', classifier2_balanced_accuracy_score)
# Print the classification report for the classifier19 model
print('\n', classifier2_imbalance_testing_report) 

In [ ]:
# Create a new empty predictions DataFrame.

# Create a predictions DataFrame
KNClassifier_predictions_df = pd.DataFrame(index = X_test.index)

# Add the SVM model predictions to the DataFrame
KNClassifier_predictions_df['Predicted'] = classifier2_pred

# Add the actual returns to the DataFrame
KNClassifier_predictions_df['Actual Returns'] = signals_df['Actual Returns']

# Add the strategy returns to the DataFrame
KNClassifier_predictions_df['Strategy Returns'] = (KNClassifier_predictions_df['Predicted'] * KNClassifier_predictions_df['Actual Returns'])

# Review the DataFrame
display(KNClassifier_predictions_df.head(3))
display(KNClassifier_predictions_df.tail(3))

In [ ]:
# Plot the actual returns versus the strategy returns
plot_4 = (1 + KNClassifier_predictions_df[['Actual Returns', 'Strategy Returns']]).cumprod().hvplot(title = 'KNeighborsClassifier - 3 month training')

___

In [ ]:
# Alternate  24 month train
# Instantiate KNN model
classifier_a_model = KNeighborsClassifier()

# fit or 'train' KNN
classifier_a_model = classifier_model.fit(Xa_train_scaled, ya_train)

# Use the testing dataset to generate the predictions for the new model
classifier_a_pred = classifier_model.predict(Xa_test_scaled)

# Review the model's predicted values
display(type(classifier_a_pred))
classifier_a_pred[:50]

In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
classifier_a_testing_report = classification_report(ya_test, classifier_a_pred)

# Print the classification report
print(classifier_a_testing_report)

In [ ]:
# Calculate the balanced_accuracy score of the model
classifier_a_balanced_accuracy_score = balanced_accuracy_score(ya_test, classifier_a_pred)

# Generate a confusion matrix for the model
classifier_a_matrix = confusion_matrix(ya_test, classifier_a_pred)

# Print the confusion matrix for the original test data
print(classifier_a_matrix)

In [ ]:
# Calculate the classification report for the model
classifier_a_imbalance_testing_report = classification_report_imbalanced(ya_test, classifier_a_pred)

In [ ]:
# Adding classes names for better interpretation
classes_names = ['Negative (-1)','Positive (+1)']
cm = pd.DataFrame(confusion_matrix(ya_test, classifier_a_pred), 
                  columns=classes_names, index = classes_names)
                  
# Seaborn's heatmap to better visualize the confusion matrix
sns.heatmap(cm, annot = True, fmt = 'd', cmap='Reds', linewidth =.5, linecolor = 'black');
# Print the balanced_accuracy score of the classifier19 model
print('\nbalanced accuracy score:', classifier_a_balanced_accuracy_score)
# Print the classification report for the classifier19 model
print('\n', classifier_a_imbalance_testing_report) 

# Finding the Best K for KNN Classification
### create a for loop and run models that have from 1 to X neighbors. At each interaction, calculate the f1 and plot the number of Ks along with the f1 result

In [ ]:
# create an empty set called 'f1s'
f1sa = []

# Calculating f1 score for K values between 1 and 40
for i in range(1, 40):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(Xa_train_scaled, ya_train)
    pred_i = knn.predict(Xa_test_scaled)
    # using average = 'weighted' to calculate a weighted average for the classes 
    f1sa.append(f1_score(ya_test, pred_i, average='weighted'))

# plot the f1 values to find the best K

In [ ]:
# plot the f1s
plt.figure(figsize=(12, 6))
plt.plot(range(1, 40), f1sa, color='red', linestyle='dashed', marker='o',
         markerfacecolor='blue', markersize = 10)
plt.title('F1 Score K Value')
plt.xlabel('K Value')
plt.ylabel('F1 Score')

In [ ]:
# New f1 plot, less data (i.e. shorter or tighter range)
plt.figure(figsize=(12, 6))
plt.plot(range(10, 40), f1sa[9:39], color='red', linestyle='dashed', marker='o',
         markerfacecolor='blue', markersize=10)
plt.title('F1 Score K Value')
plt.xlabel('K Value')
plt.ylabel('F1 Score')

In [ ]:
# New f1 plot, less data (i.e. shorter or tighter range)
plt.figure(figsize=(12, 6))
plt.plot(range(30, 40), f1sa[29:39], color='red', linestyle='dashed', marker='o',
         markerfacecolor='blue', markersize=10)
plt.title('F1 Score K Value')
plt.xlabel('K Value')
plt.ylabel('F1 Score')

In [ ]:
# obtain the highest f1 using the built-in max()
print(max(f1sa))                       # 0.5094344985062002
# convert the list into a NumPy array and use argmax() to get highest index of that point
print(np.array(f1sa).argmax())         # 31, remember zero indexing

In [ ]:
# Instantiate new KNN model
classifier2_a_model = KNeighborsClassifier(n_neighbors = 32)          

# fit or 'train' KNN
classifier2_a_model = classifier2_a_model.fit(Xa_train_scaled, ya_train)

# Use the testing dataset to generate the predictions for the new model
classifier2_a_pred = classifier2_model.predict(Xa_test_scaled)

# Review the model's predicted values
display(type(classifier2_a_pred))
classifier2_a_pred[:50]

In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
classifier2_a_testing_report = classification_report(ya_test, classifier2_a_pred)

# Print the classification report
print(classifier2_a_testing_report)

In [ ]:
# Calculate the balanced_accuracy score of the model
classifier2_a_balanced_accuracy_score = balanced_accuracy_score(ya_test, classifier2_a_pred)

# Generate a confusion matrix for the model
classifier2_a_matrix = confusion_matrix(ya_test, classifier2_a_pred)

# Print the confusion matrix for the original test data
print(classifier2_a_matrix)

In [ ]:
# Calculate the classification report for the model
classifier2_a_imbalance_testing_report = classification_report_imbalanced(ya_test, classifier2_a_pred)

In [ ]:
# Adding classes names for better interpretation
classes_names = ['Negative (-1)','Positive (+1)']
cm = pd.DataFrame(confusion_matrix(ya_test, classifier2_a_pred), 
                  columns=classes_names, index = classes_names)
                  
# Seaborn's heatmap to better visualize the confusion matrix
sns.heatmap(cm, annot = True, fmt = 'd', cmap='Reds', linewidth =.5, linecolor = 'black');
# Print the balanced_accuracy score of the classifier19 model
print('\nbalanced accuracy score:', classifier2_a_balanced_accuracy_score)
# Print the classification report for the classifier19 model
print('\n', classifier2_a_imbalance_testing_report) 

In [ ]:
# Create a new empty predictions DataFrame.

# Create a predictions DataFrame
KNClassifier_a_predictions_df = pd.DataFrame(index = Xa_test.index)

# Add the SVM model predictions to the DataFrame
KNClassifier_a_predictions_df['Predicted'] = classifier2_a_pred

# Add the actual returns to the DataFrame
KNClassifier_a_predictions_df['Actual Returns'] = signals_df['Actual Returns']

# Add the strategy returns to the DataFrame
KNClassifier_a_predictions_df['Strategy Returns'] = (KNClassifier_a_predictions_df['Predicted'] * KNClassifier_a_predictions_df['Actual Returns'])

# Review the DataFrame
display(KNClassifier_a_predictions_df.head(3))
display(KNClassifier_a_predictions_df.tail(3))

In [ ]:
# Plot the actual returns versus the strategy returns
plot_4a = (1 + KNClassifier_a_predictions_df[['Actual Returns', 'Strategy Returns']]).cumprod().hvplot(title = 'KNeighborsClassifier - 24 month training')
plot_4 + plot_4a